In [ ]:

from xmitgcm import llcreader
model = llcreader.ECCOPortalLLC4320Model()
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import xrft
import xgcm
import gcm_filters
import matplotlib.pyplot as plt
import xesmf as xe
import gsw_xarray as gsw


# In[ ]:


# load full temperature and salinity datasets
ds_T_full = model.get_dataset(varnames=['Theta'], type='latlon')
ds_S_full = model.get_dataset(varnames=['Salt'], type='latlon')


# In[ ]:


# load full velocity dataset 
ds_UV_full = model.get_dataset(varnames=['U','V'], type='latlon')
ds_W_full = model.get_dataset(varnames=['W'], type='latlon')

# interp full velocities onto tracer grid
ds_W_full_interp = ds_W_full.interp(k_l=ds_T_full.k)
ds_UV_full_interp = ds_UV_full.interp(j_g=ds_T_full.j, i_g=ds_T_full.i)

del ds_UV_full, ds_W_full

# In[12]:


# select subdomain area and downsample every 24 hours to reduce auto-correlation between samples (Uchida et. al. 2022)
# depth above -1000m (as we will average over the mixed layer below)
# lat= 25:40, lon=-72:-58 # change domain to be larger than 12 degrees

lat_min = -57
lat_max = -42
lon_min = 23
lon_max = 37
depth_lim = -1000 

sel_area = np.logical_and(np.logical_and(np.logical_and(ds_T_full.XC>lon_min, ds_T_full.XC<lon_max ),
                           np.logical_and(ds_T_full.YC>lat_min, ds_T_full.YC<lat_max)), ds_T_full.Z>depth_lim)

# temp, salt and velocity in subdomain
T = ds_T_full.Theta.resample(time='24H').nearest(tolerance="1H").where(sel_area, drop=True)
S = ds_S_full.Salt.resample(time='24H').nearest(tolerance="1H").where(sel_area, drop=True)
U = ds_UV_full_interp.U.resample(time='24H').nearest(tolerance="1H").where(sel_area, drop=True)
V = ds_UV_full_interp.V.resample(time='24H').nearest(tolerance="1H").where(sel_area, drop=True)
W = ds_W_full_interp.W.resample(time='24H').nearest(tolerance="1H").where(sel_area, drop=True)

del ds_T_full, ds_S_full, ds_UV_full_interp, ds_W_full_interp

# In[13]:


# calculate buoyancy from temp and salt, using the gsw package
# pressure
P = gsw.p_from_z(T.Z, T.YC)

# gravity
G = gsw.grav(T.YC, P)

# Conservative Temperature (CT)
CT = gsw.CT_from_pt(S, T)

# potential density anomaly 
# with the reference pressure of 0 dbar and ρ0 = 1000 kg m−3
sigma0 = gsw.sigma0(S, CT)

# reference density 
rho0 = 1000 #kg/m^3

# buoyancy
B = -G*sigma0/rho0

B = B.rename('Buoyancy')

del S, P, G, CT

# In[14]:
# regrid and select middle 10 degree box
lon_mid = (lon_min + lon_max)/2
lat_mid = (lat_min + lat_max)/2

lon_slice_10deg = slice(lon_mid-5,lon_mid+5)
lat_slice_10deg = slice(lat_mid-5,lat_mid+5)

sel_area_10deg = np.where(np.logical_and(np.logical_and(T.XC>lon_mid-5, T.XC<lon_mid+5 ),
                           np.logical_and(T.YC>lat_mid-5, T.YC<lat_mid+5)))

i_min_10deg = min(sel_area_10deg[1])
i_max_10deg = max(sel_area_10deg[1])
j_min_10deg = min(sel_area_10deg[0]) 
j_max_10deg = max(sel_area_10deg[0])


i_slice_10deg = slice(i_min_10deg, i_max_10deg)
j_slice_10deg = slice(j_min_10deg, j_max_10deg)

B.isel(time=0,k=0,i=i_slice_10deg,j=j_slice_10deg).plot()
plt.savefig('B.png')
plt.close()

U.isel(time=0,k=0,i=i_slice_10deg,j=j_slice_10deg).plot()
plt.savefig('U.png')
plt.close()

V.isel(time=0,k=0,i=i_slice_10deg,j=j_slice_10deg).plot()
plt.savefig('V.png')
plt.close()

W.isel(time=0,k=0,i=i_slice_10deg,j=j_slice_10deg).plot()
plt.savefig('W.png')
plt.close()
# In[15]:


# sigma0 at 10m depth for reference
sigma0_10m = sigma0.isel(k=6).broadcast_like(sigma0).chunk(chunks={'time': 1, 'k':1, 'j': sigma0.j.size, 'i': sigma0.i.size})

delta_sigma = sigma0 - sigma0_10m

# mixed layer depth
MLD = sigma0.Z.broadcast_like(sigma0).where(delta_sigma<=0.03).min(dim="k",skipna=True).chunk(chunks={'time': 1, 'j': sigma0.j.size, 'i': sigma0.i.size}).rename('Mixed Layer Depth')


# In[17]:

MLD.isel(time=0,i=i_slice_10deg,j=j_slice_10deg).plot()
plt.savefig('MLD.png')
plt.close()
# # Filtering: fixed factor scale 24

# In[18]:


gcm_filters.required_grid_vars(gcm_filters.GridType.IRREGULAR_WITH_LAND)


# In[19]:


# define input parameters

# wet mask
wet_mask = xr.DataArray(np.logical_not(np.isnan(T.Depth)).values,coords={'j':T.j.values , 'i': T.i.values})

# area
area= T.rA

#grid
dxw = xr.DataArray(
    data=U.dxC, 
    coords={'i':T.i,'j':T.j}, 
    dims=('j','i')
)

dyw = xr.DataArray(
    data=U.dyC, 
    coords={'i':T.i,'j':T.j}, 
    dims=('j','i')
)


dxs = xr.DataArray(
    data=U.dxG, 
    coords={'i':T.i,'j':T.j}, 
    dims=('j','i')
)


dys = xr.DataArray(
    data=U.dyG, 
    coords={'i':T.i,'j':T.j}, 
    dims=('j','i')
)

# dx_min
dx_min = min(dxw.min(),dyw.min(),dxs.min(),dys.min())
dx_min = dx_min.values
dx_min

#dx_max
dx_max = max(dxw.max(),dys.max()) #removed ,dyw.max(),dxs.max()
dx_max = dx_max.values
dx_max

#kappa_w and kappa_s
kappa_w_fac = dxw * dxw / (dx_max * dx_max) 
kappa_s_fac = dys * dys / (dx_max * dx_max)


# In[20]:


# define filter
filter_factor24 = gcm_filters.Filter(
    filter_scale=24 * dx_max, # factor of 24
    dx_min=dx_min,
    filter_shape=gcm_filters.FilterShape.GAUSSIAN,
    grid_type=gcm_filters.GridType.IRREGULAR_WITH_LAND,
    grid_vars={
        'wet_mask': wet_mask, 
        'dxw': dxw, 'dyw': dyw, 'dxs': dxs, 'dys': dys, 'area': area, 
        'kappa_w': kappa_w_fac, 'kappa_s': kappa_s_fac
    }
)
filter_factor24


# ## Define Mesoscale and submesoscale fields

# In[21]:


# mesoscale field defined by the 24 factor filter

Um = filter_factor24.apply(U.where(wet_mask), dims=['i', 'j'])
Vm = filter_factor24.apply(V.where(wet_mask), dims=['i', 'j'])
Wm = filter_factor24.apply(W.where(wet_mask), dims=['i', 'j'])
Bm = filter_factor24.apply(B.where(wet_mask), dims=['i', 'j'])


# In[3]:





# In[22]:

Bm.isel(time=0,k=0,i=i_slice_10deg,j=j_slice_10deg).plot()
plt.savefig('Bm.png')
plt.close()

Um.isel(time=0,k=0,i=i_slice_10deg,j=j_slice_10deg).plot()
plt.savefig('Um.png')
plt.close()

Vm.isel(time=0,k=0,i=i_slice_10deg,j=j_slice_10deg).plot()
plt.savefig('Vm.png')
plt.close()

Wm.isel(time=0,k=0,i=i_slice_10deg,j=j_slice_10deg).plot()
plt.savefig('Wm.png')
plt.close()
# In[23]:

# submesoscale field defined as the deviation between the full resolved field and mesoscale field

Us = U - Um
Vs = V - Vm
Ws = W - Wm
Bs = B - Bm

del U, V, W, B, MLD

# In[24]:

Bs.isel(time=0,k=0,i=i_slice_10deg,j=j_slice_10deg).plot()
plt.savefig('Bs.png')
plt.close()

Us.isel(time=0,k=0,i=i_slice_10deg,j=j_slice_10deg).plot()
plt.savefig('Us.png')
plt.close()

Vs.isel(time=0,k=0,i=i_slice_10deg,j=j_slice_10deg).plot()
plt.savefig('Vs.png')
plt.close()

Ws.isel(time=0,k=0,i=i_slice_10deg,j=j_slice_10deg).plot()
plt.savefig('Ws.png')
plt.close()
# In[ ]:

# submesoscale buoyancy fluxes

UsBs = Us * Bs
VsBs = Vs * Bs
WsBs = Ws * Bs



# In[ ]:

UsBs.isel(time=0,k=0,i=i_slice_10deg,j=j_slice_10deg).plot()
plt.savefig('UsBs.png')
plt.close()

VsBs.isel(time=0,k=0,i=i_slice_10deg,j=j_slice_10deg).plot()
plt.savefig('VsBs.png')
plt.close()

WsBs.isel(time=0,k=0,i=i_slice_10deg,j=j_slice_10deg).plot()
plt.savefig('WsBs.png')
plt.close()
# In[ ]:

del Us, Vs, Ws, Bs 


# mesoscale buoyancy gradient
# need to tomperarily rechunk for the z derivative
Bm_x = Bm.differentiate("i")
Bm_y = Bm.differentiate("j")
Bm_z = Bm.chunk(chunks={"time":len(T.time), "k":len(T.k), "j":len(T.j), "i": len(T.i)}).differentiate("k").chunk(chunks={"time":1, "k":1, "j":len(T.j), "i": len(T.i)})

# In[ ]:
Bm_x.isel(time=0,k=0,i=i_slice_10deg,j=j_slice_10deg).plot()
plt.savefig('Bm_x.png')
plt.close()

Bm_y.isel(time=0,k=0,i=i_slice_10deg,j=j_slice_10deg).plot()
plt.savefig('Bm_y.png')
plt.close()

Bm_z.isel(time=0,k=0,i=i_slice_10deg,j=j_slice_10deg).plot()
plt.savefig('Bm_z.png')
plt.close()

# ## Coarse-grain to 1/4 degree
# 
# 

# In[ ]:


# building regridder: coarse grain to 1/4 degree

ds_out_qurter = xr.Dataset(
    {
        "lat": (["lat"], np.arange(lat_min, lat_max, 0.25)),
        "lon": (["lon"], np.arange(lon_min, lon_max, 0.25)),
    }
)

regridder_quarter = xe.Regridder(T.isel(time=0, k=0), ds_out_qurter, "bilinear", periodic=False, reuse_weights=False)
regridder_quarter


# In[ ]:


# regrid and select middle 10 degree box
lon_mid = (lon_min + lon_max)/2
lat_mid = (lat_min + lat_max)/2

lon_slice_10deg = slice(lon_mid-5,lon_mid+5)
lat_slice_10deg = slice(lat_mid-5,lat_mid+5)

# meso velocities and buoyancy
Um_lowres = regridder_quarter(Um).sel(lon=lon_slice_10deg,lat=lat_slice_10deg) 
Vm_lowres = regridder_quarter(Vm).sel(lon=lon_slice_10deg,lat=lat_slice_10deg) 
Wm_lowres = regridder_quarter(Wm).sel(lon=lon_slice_10deg,lat=lat_slice_10deg) 
Bm_lowres = regridder_quarter(Bm).sel(lon=lon_slice_10deg,lat=lat_slice_10deg) 

# meso gradients
Bm_x_lowres = regridder_quarter(Bm_x).sel(lon=lon_slice_10deg,lat=lat_slice_10deg) 
Bm_y_lowres = regridder_quarter(Bm_y).sel(lon=lon_slice_10deg,lat=lat_slice_10deg) 
Bm_z_lowres = regridder_quarter(Bm_z).sel(lon=lon_slice_10deg,lat=lat_slice_10deg) 

# submeso fluxes
UsBs_lowres = regridder_quarter(UsBs).sel(lon=lon_slice_10deg,lat=lat_slice_10deg)  
VsBs_lowres = regridder_quarter(VsBs).sel(lon=lon_slice_10deg,lat=lat_slice_10deg) 
WsBs_lowres = regridder_quarter(WsBs).sel(lon=lon_slice_10deg,lat=lat_slice_10deg)  

del Um, Vm, Wm, Bm, UsBs, VsBs, WsBs, Bm_x, Bm_y, Bm_z

# In[ ]:

Bm_lowres.isel(time=0,k=0).plot()
plt.savefig('Bm_lowres.png')
plt.close()

Um_lowres.isel(time=0,k=0).plot()
plt.savefig('Um_lowres.png')
plt.close()

Vm_lowres.isel(time=0,k=0).plot()
plt.savefig('Vm_lowres.png')
plt.close()

Wm_lowres.isel(time=0,k=0).plot()
plt.savefig('Wm_lowres.png')
plt.close()

Bm_x_lowres.isel(time=0,k=0).plot()
plt.savefig('Bm_x_lowres.png')
plt.close()

Bm_y_lowres.isel(time=0,k=0).plot()
plt.savefig('Bm_y_lowres.png')
plt.close()

Bm_z_lowres.isel(time=0,k=0).plot()
plt.savefig('Bm_z_lowres.png')
plt.close()

UsBs_lowres.isel(time=0,k=0).plot()
plt.savefig('UsBs_lowres.png')
plt.close()

VsBs_lowres.isel(time=0,k=0).plot()
plt.savefig('VsBs_lowres.png')
plt.close()

WsBs_lowres.isel(time=0,k=0).plot()
plt.savefig('WsBs_lowres.png')
plt.close()

del Um_lowres, Vm_lowres, Wm_lowres, Bm_lowres
del UsBs_lowres, VsBs_lowres, WsBs_lowres
del Bm_x_lowres, Bm_y_lowres, Bm_z_lowres

# In[ ]:


# meso velocities and buoyancy
Um_lowres_MLD = Um_lowres.where(delta_sigma<=0.03).min(dim="k",skipna=True).rename('Um_lowres_MLD')
Vm_lowres_MLD = Vm_lowres.where(delta_sigma<=0.03).min(dim="k",skipna=True).rename('Vm_lowres_MLD')
Wm_lowres_MLD = Wm_lowres.where(delta_sigma<=0.03).min(dim="k",skipna=True).rename('Wm_lowres_MLD')
Bm_lowres_MLD = Bm_lowres.where(delta_sigma<=0.03).min(dim="k",skipna=True).rename('Bm_lowres_MLD')

# submeso fluxes 
UsBs_lowres_MLD = UsBs_lowres.where(delta_sigma<=0.03).min(dim="k",skipna=True).rename('UsBs_lowres_MLD')
VsBs_lowres_MLD = VsBs_lowres.where(delta_sigma<=0.03).min(dim="k",skipna=True).rename('VsBs_lowres_MLD')
WsBs_lowres_MLD = WsBs_lowres.where(delta_sigma<=0.03).min(dim="k",skipna=True).rename('WsBs_lowres_MLD')

# meso gradients
Bm_x_lowres_MLD = Bm_x_lowres.where(delta_sigma<=0.03).min(dim="k",skipna=True).rename('Bm_x_lowres_MLD')
Bm_y_lowres_MLD = Bm_y_lowres.where(delta_sigma<=0.03).min(dim="k",skipna=True).rename('Bm_y_lowres_MLD')
Bm_z_lowres_MLD = Bm_z_lowres.where(delta_sigma<=0.03).min(dim="k",skipna=True).rename('Bm_z_lowres_MLD')


# In[ ]:

Bm_lowres_MLD.isel(time=0).plot()
plt.savefig('Bm_lowres_MLD.png')
plt.close()

Um_lowres_MLD.isel(time=0).plot()
plt.savefig('Um_lowres_MLD.png')
plt.close()

Vm_lowres_MLD.isel(time=0).plot()
plt.savefig('Vm_lowres_MLD.png')
plt.close()

Wm_lowres_MLD.isel(time=0).plot()
plt.savefig('Wm_lowres_MLD.png')
plt.close()

Bm_x_lowres_MLD.isel(time=0).plot()
plt.savefig('Bm_x_lowres_MLD.png')
plt.close()

Bm_y_lowres_MLD.isel(time=0).plot()
plt.savefig('Bm_y_lowres_MLD.png')
plt.close()

Bm_z_lowres_MLD.isel(time=0,k=0).plot()
plt.savefig('Bm_z_lowres_MLD.png')
plt.close()

UsBs_lowres_MLD.isel(time=0).plot()
plt.savefig('UsBs_lowres_MLD.png')
plt.close()

VsBs_lowres_MLD.isel(time=0).plot()
plt.savefig('VsBs_lowres_MLD.png')
plt.close()

WsBs_lowres_MLD.isel(time=0).plot()
plt.savefig('WsBs_lowres_MLD.png')
plt.close()
# # Save data: stack input and output, save snapshots

# In[ ]:


def save_data(PATH, CASE_NAME, dataset_ind, Um_lowres_MLD, Vm_lowres_MLD, Wm_lowres_MLD, Bm_lowres_MLD,UsBs_lowres_MLD, VsBs_lowres_MLD, WsBs_lowres_MLD, Bm_x_lowres_MLD, Bm_y_lowres_MLD, Bm_z_lowres_MLD):
    
    for ii in tqdm(range(len(dataset_ind))):
        # input channels: lowres mesoscale fields averaged over the mixed layer Um, Vm, Wm, Bm
        Um_data = np.nan_to_num(Um_lowres_MLD.Um_lowres_MLD.isel(time=dataset_ind[ii]).values)
        Vm_data = np.nan_to_num(Vm_lowres_MLD.Vm_lowres_MLD.isel(time=dataset_ind[ii]).values)
        Wm_data = np.nan_to_num(Wm_lowres_MLD.Wm_lowres_MLD.isel(time=dataset_ind[ii]).values)
        Bm_data = np.nan_to_num(Bm_lowres_MLD.Bm_lowres_MLD.isel(time=dataset_ind[ii]).values)
        
        # output channels: lowres submeso fluxes and meso gradient fields averaged over the mixed layer UsBs, VsBs, WsBs, Bm_x, Bm_y, Bm_z
        UsBs_data = np.nan_to_num(UsBs_lowres_MLD.UsBs_lowres_MLD.isel(time=dataset_ind[ii]).values)
        VsBs_data = np.nan_to_num(VsBs_lowres_MLD.VsBs_lowres_MLD.isel(time=dataset_ind[ii]).values)
        WsBs_data = np.nan_to_num(WsBs_lowres_MLD.WsBs_lowres_MLD.isel(time=dataset_ind[ii]).values)
        
        Bm_x_data = np.nan_to_num(Bm_x_lowres_MLD.Bm_x_lowres_MLD.isel(time=dataset_ind[ii]).values)
        Bm_y_data = np.nan_to_num(Bm_y_lowres_MLD.Bm_y_lowres_MLD.isel(time=dataset_ind[ii]).values)
        Bm_z_data = np.nan_to_num(Bm_z_lowres_MLD.Bm_z_lowres_MLD.isel(time=dataset_ind[ii]).values)
        

        # input: 
        cnn_input_ii = np.stack([
            Um_data,
            Vm_data,
            Wm_data,
            Bm_data
        ], axis=0).astype(np.float32)
        
        #output
        cnn_output_ii = np.stack([
            UsBs_data,
            VsBs_data,
            WsBs_data,
            Bm_x_data,
            Bm_y_data,
            Bm_z_data
        ], axis=0).astype(np.float32)


    
    
        ########## Save  data
        FULL_PATH_input = PATH+CASE_NAME+'input/'
        FULL_PATH_output = PATH+CASE_NAME+'output/'

        # Check whether the specified path exists or not
        isExist = os.path.exists(FULL_PATH_input)
        if not isExist:
            os.makedirs(FULL_PATH_input)
            print("The new input directory is created!")
            os.makedirs(FULL_PATH_output)
            print("The new output directory is created!")
        

        np.save(FULL_PATH_input+str(ii)+'.npy', cnn_input_ii)
        np.save(FULL_PATH_output+str(ii)+'.npy', cnn_output_ii)

    np.save(PATH+CASE_NAME+'dataset_ind.npy', dataset_ind)


# In[ ]:


# randomnly generate test and training time indecies (need to make sure no overlapping times)

time_ind = len(T.time)
rand_ind = np.arange(time_ind)
np.random.shuffle(rand_ind)
train_percent = 0.7
test_percent = 0.2 
print(f"Dataset: train {np.round(train_percent*100)}%, test {np.round(test_percent*100)}%, val {np.round((1-train_percent-test_percent)*100)}%")
train_ind, test_ind, val_ind =  rand_ind[:round(train_percent*time_ind)], rand_ind[round(train_percent*time_ind):round((train_percent+test_percent)*time_ind)], rand_ind[round((train_percent+test_percent)*time_ind):]                                                                        


# In[ ]:


# Save train, test and val datasets
CASE_NAME = 'lat_-57_-42_lon_25_40'
BASE = '/scratch/ab10313/submeso_ML_data/'

# Train set
print('TRAIN: START')
TRAIN_PATH = BASE+'train/'
save_data(TRAIN_PATH, CASE_NAME, train_ind, Um_lowres_MLD, Vm_lowres_MLD, Wm_lowres_MLD, Bm_lowres_MLD,UsBs_lowres_MLD, VsBs_lowres_MLD, WsBs_lowres_MLD, Bm_x_lowres_MLD, Bm_y_lowres_MLD, Bm_z_lowres_MLD)
print('DONE')

# TEST set
print('TEST: START')
TEST_PATH = BASE+'test/'
save_data(TEST_PATH, CASE_NAME, test_ind,  Um_lowres_MLD, Vm_lowres_MLD, Wm_lowres_MLD, Bm_lowres_MLD,UsBs_lowres_MLD, VsBs_lowres_MLD, WsBs_lowres_MLD, Bm_x_lowres_MLD, Bm_y_lowres_MLD, Bm_z_lowres_MLD)
print('DONE')

# VAL set
print('VAL: START')
VAL_PATH = BASE+'val/'
save_data(VAL_PATH, CASE_NAME, val_ind, Um_lowres_MLD, Vm_lowres_MLD, Wm_lowres_MLD, Bm_lowres_MLD,UsBs_lowres_MLD, VsBs_lowres_MLD, WsBs_lowres_MLD, Bm_x_lowres_MLD, Bm_y_lowres_MLD, Bm_z_lowres_MLD)
print('DONE')


print('COMPLETE')


In [1]:

from xmitgcm import llcreader
model = llcreader.ECCOPortalLLC4320Model()
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import xrft
import xgcm
import gcm_filters
import matplotlib.pyplot as plt
import xesmf as xe
import gsw_xarray as gsw

In [2]:
ds_T_full = model.get_dataset(varnames=['Theta'], type='latlon')

In [34]:

lat_min = -57
lat_max = -42
lon_min = 23
lon_max = 37
depth_lim = -1000 

sel_area = np.logical_and(np.logical_and(np.logical_and(ds_T_full.XC>lon_min, ds_T_full.XC<lon_max ),
                           np.logical_and(ds_T_full.YC>lat_min, ds_T_full.YC<lat_max)), ds_T_full.Z>depth_lim)

# temp, salt and velocity in subdomain
T = ds_T_full.Theta.resample(time='24H').nearest(tolerance="1H").where(sel_area, drop=True)

KeyboardInterrupt: 

In [36]:
sel_area_10deg

NameError: name 'sel_area_10deg' is not defined

In [4]:
fig = T.isel(time=0,k=0).plot()
plt.savefig('T_figtest_var.png')
plt.close()

In [37]:
lon_mid = (lon_min + lon_max)/2
lat_mid = (lat_min + lat_max)/2

lon_slice_10deg = slice(lon_mid-5,lon_mid+5)
lat_slice_10deg = slice(lat_mid-5,lat_mid+5)

sel_area_10deg = np.where(np.logical_and(np.logical_and(T.XC>lon_mid-5, T.XC<lon_mid+5 ),
                           np.logical_and(T.YC>lat_mid-5, T.YC<lat_mid+5)))

i_min_10deg = min(sel_area_10deg[1])
i_max_10deg = max(sel_area_10deg[1])
j_min_10deg = min(sel_area_10deg[0]) 
j_max_10deg = max(sel_area_10deg[0])


i_slice_10deg = slice(i_min_10deg, i_max_10deg)
j_slice_10deg = slice(j_min_10deg, j_max_10deg)

In [50]:
lon_vals = T.XC.isel(i=i_slice_10deg,j=j_slice_10deg)[0,:].values
lat_vals = T.YC.isel(i=i_slice_10deg,j=j_slice_10deg)[:,0].values

In [51]:
lat_vals

array([-54.496284, -54.484722, -54.47316 , -54.461594, -54.450024,
       -54.438454, -54.42688 , -54.415302, -54.40372 , -54.39214 ,
       -54.380554, -54.368965, -54.357372, -54.34578 , -54.334183,
       -54.322582, -54.31098 , -54.299374, -54.287766, -54.276157,
       -54.26454 , -54.252926, -54.241306, -54.229683, -54.21806 ,
       -54.206432, -54.1948  , -54.18317 , -54.17153 , -54.159893,
       -54.148254, -54.13661 , -54.12496 , -54.11331 , -54.101658,
       -54.090004, -54.078346, -54.066685, -54.05502 , -54.043354,
       -54.031685, -54.02001 , -54.008335, -53.99666 , -53.984978,
       -53.973293, -53.96161 , -53.949917, -53.938225, -53.926533,
       -53.914833, -53.903133, -53.89143 , -53.879723, -53.868015,
       -53.856304, -53.84459 , -53.83287 , -53.82115 , -53.80943 ,
       -53.797703, -53.785976, -53.774242, -53.76251 , -53.750774,
       -53.739033, -53.72729 , -53.715546, -53.703796, -53.692047,
       -53.680294, -53.668537, -53.656776, -53.645016, -53.633

In [31]:
T_z_test = T.chunk(chunks={"time":len(T.time), "k":len(T.k), "j":len(T.j), "i": len(T.i)}).differentiate("k").chunk(chunks={"time":1, "k":len(T.k), "j":len(T.j), "i": len(T.i)})


In [6]:
del fig

In [23]:
T.isel(time=0)

<xarray.DataArray 'Theta' (k: 52, j: 1186, i: 672)>
dask.array<getitem, shape=(52, 1186, 672), dtype=float32, chunksize=(1, 1186, 672), chunktype=numpy.ndarray>
Coordinates:
    time     datetime64[ns] 2011-09-13
  * i        (i) int64 2928 2929 2930 2931 2932 ... 3595 3596 3597 3598 3599
  * j        (j) int64 4321 4322 4323 4324 4325 ... 5502 5503 5504 5505 5506
  * k        (k) int64 0 1 2 3 4 5 6 7 8 9 10 ... 42 43 44 45 46 47 48 49 50 51
    CS       (j, i) float32 dask.array<chunksize=(1186, 672), meta=np.ndarray>
    SN       (j, i) float32 dask.array<chunksize=(1186, 672), meta=np.ndarray>
    Depth    (j, i) float32 dask.array<chunksize=(1186, 672), meta=np.ndarray>
    drF      (k) >f4 1.0 1.14 1.3 1.49 1.7 ... 42.62 43.73 44.87 46.05 47.28
    dxF      (j, i) float32 dask.array<chunksize=(1186, 672), meta=np.ndarray>
    dyF      (j, i) float32 dask.array<chunksize=(1186, 672), meta=np.ndarray>
    hFacC    (k, j, i) float32 dask.array<chunksize=(1, 1186, 672), meta=np.ndarray>
    PHrefC   (k) >f4 4.905 15.4 27.37 41.05 ... 8.83e+03 9.276e+03 9.734e+03
    rA       (j, i) float32 dask.array<chunksize=(1186, 672), meta=np.ndarray>
    Z        (k) >f4 -0.5 -1.57 -2.79 -4.185 ... -855.8 -900.1 -945.6 -992.3
    rhoRef   (k) >f4 1.025e+03 1.025e+03 1.025e+03 ... 1.032e+03 1.032e+03
    XC       (j, i) float32 dask.array<chunksize=(1186, 672), meta=np.ndarray>
    YC       (j, i) float32 dask.array<chunksize=(1186, 672), meta=np.ndarray>
Attributes:
    standard_name:  THETA
    long_name:      Potential Temperature
    units:          degC

In [54]:
def save_snapshot(PATH, var, var_save_name):

    # snapshot values
    var_values = var.values
    
    ########## Save  data
    FULL_PATH = PATH+'/snapshots/'

    # Check whether the specified path exists or not
    isExist = os.path.exists(FULL_PATH)
    if not isExist:
        os.makedirs(FULL_PATH)
        print("The new snapshots directory is created!")

    np.save(FULL_PATH+var_save_name+'.npy', var_values)
    del var_values

In [55]:
CASE_NAME = 'lat_-57_-42_lon_23_37/'
BASE = '/scratch/ab10313/submeso_ML_data/'

PATH = BASE+CASE_NAME

save_snapshot(PATH,T.XC.isel(i=i_slice_10deg,j=j_slice_10deg)[0,:],'lon')
save_snapshot(PATH,T.YC.isel(i=i_slice_10deg,j=j_slice_10deg)[:,0],'lat')
#save_snapshot(PATH, T.isel(k=0,time=0), 'T_test_sanpshot')

In [56]:
ds_out_qurter = xr.Dataset(
    {
        "lat": (["lat"], np.arange(lat_min, lat_max, 0.25)),
        "lon": (["lon"], np.arange(lon_min, lon_max, 0.25)),
    }
)

In [67]:
lon_mid = (lon_min + lon_max)/2
lat_mid = (lat_min + lat_max)/2

lon_slice_10deg = slice(lon_mid-5,lon_mid+5)
lat_slice_10deg = slice(lat_mid-5,lat_mid+5)

#np.save(PATH+'snapshots/lat_lowres.npy',np.arange(lat_mid-5, lat_mid+5, 0.25))
#np.save(PATH+'snapshots/lon_lowres.npy',np.arange(lon_mid-5, lon_mid+5, 0.25))

In [105]:
lat_lowres = np.arange(lat_min, lat_max, 0.25)
lat_lowres = lat_lowres[np.where(np.logical_and(lat_lowres>=lat_mid-5,lat_lowres<=lat_mid+5))]

lon_lowres = np.arange(lon_min, lon_max, 0.25)
lon_lowres = lon_lowres[np.where(np.logical_and(lon_lowres>=lon_mid-5,lon_lowres<=lon_mid+5))]

np.save(PATH+'snapshots/lat_lowres.npy',lon_lowres)
np.save(PATH+'snapshots/lon_lowres.npy',lat_lowres)

In [106]:
lat_lowres.size

41

In [107]:
lon_lowres.size

41